In [8]:
#Create a function
import nibabel as nib
import nilearn
from nilearn import image
from moviepy.editor import *
import numpy as np
from scipy.stats import zscore
import h5py
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import os
import pandas as pd
import deepdish as dd
from sklearn import linear_model
import math
import sys
from brainiak import image, io
from brainiak.isc import isc, isfc, permutation_isc

#script_dir = '/home/msachs/my_libraries/'
#if script_dir not in sys.path:
    #sys.path.append(script_dir)
#import hmm_functions
#from hmm_functions import make_conf
#from importlib import reload
#reload(hmm_functions)

#check different templates
# for f in [elem for elem in os.listdir('/home/msachs/templateflow') if 'nii.gz' in elem]:
#     temp = nib.load('/home/msachs/templateflow/%s' %f)
#     #temp = np.nonzero(nib.load('/home/msachs/templateflow/%s' %f).get_data() > 0)
#     print(temp.shape)

# for f in [elem for elem in os.listdir('/home/msachs/templateflow/ROIs_for_TFA/') if 'nii' in elem]:
#     temp = nib.load('/home/msachs/templateflow/ROIs_for_TFA/%s' %f)
#     #temp = np.nonzero(nib.load('/home/msachs/templateflow/%s' %f).get_data() > 0)
#     print(f,temp.shape)

# Hemodynamic Response Function (from AFNI)
dt = np.arange(0, 15)
p = 8.6
q = 0.547
hrf = np.power(dt / (p * q), p) * np.exp(p - dt / q)
TR = 1

def rms_reg(audio_file,nTR,ignore_TRs):
    # Load audio, and calculate audio envelope regressor
    print('Loading',audio_file)
    clip = AudioFileClip(audio_file)
    Fs = clip.fps
    samples = clip.to_soundarray()
    samples = np.mean(samples, axis=1)
    T = np.floor(samples.shape[0] / Fs).astype(int)
    rms = np.zeros(T)
    for t in range(T):
        rms[t] = np.sqrt(np.mean(np.power(samples[(Fs * t):(Fs * (t + 1))], 2)))
    rms_conv = np.convolve(rms, hrf)[:T]
    rms_TRs = zscore(np.interp(np.linspace(0, (nTR-ignore_TRs - 1) * TR, nTR-ignore_TRs),
                               np.arange(0, T), rms_conv), ddof=1) #FIXED TO INCLUDE IGNORE_TRS
    return rms_TRs, rms_conv, rms



def audcor(subj_format,conffile,rms_TRs,ignore_TRs,cc_win):
    
    # Get metadata about fMRI data
    nii_template = nib.load(subj_format)
    TR = nii_template.header['pixdim'][4]
    nii_shape = nii_template.header.get_data_shape()
    nTR = nii_shape[3]

    # Compute correlation between rms_TRs and each voxel timecourse
    print('Loading brain and masking...')
    D = nib.load(subj_format).get_data()
    a1_mask = D[aud_bool]
    
    reg = make_conf2(conffile)
    regr = linear_model.LinearRegression()
    regr.fit(reg, a1_mask.T)
    a1_clean = a1_mask - np.dot(regr.coef_, reg.T) - regr.intercept_[:, np.newaxis]

    print('Cleaning and zscoring',reg.shape)
    #a1_clean2 = nilearn.image.clean_img(a1_mask, sessions=None, detrend=False, standardize=False, confounds=reg, low_pass=None, high_pass=None, t_r=None, ensure_finite=False, mask_img=None)

    a1_cut = a1_clean[:,startcut:(nTR -endcut)]
    a1_zscore = zscore(a1_cut, axis=1)
    a1 = np.mean(a1_zscore,axis = 0)
    
    print('RMS',rms_TRs.shape,'A1',a1.shape,'num_TRs',nTR,a1.max())
    #PLOT 
    f, ax = plt.subplots(1, 1,figsize=(8,3))
    ax.plot(zscore(a1))
    ax.plot(zscore(rms_TRs))
    ax.set_title('%s-%s' %(subj_format.split('/')[5],audio_file.split('/')[5]))
    f.savefig('/data/musevent/aud_rms_check/%s_%s_a1_rmsTRs.png' %(subj_format.split('/')[5],audio_file.split('/')[5][:-4]))

    # Compute A1 cross-correlation with audio
    print('Calculating correlations...')
    cc = np.zeros(2 * cc_win + 1)
    a1_mid = a1[(cc_win):(-cc_win)]
    for i in range(2 * cc_win + 1):
        if i == 2 * cc_win:
            audio_shift = rms_TRs[i:]
        else:
            audio_shift = rms_TRs[i:(-2 * cc_win + i)]
        cc[i] = pearsonr(audio_shift, a1_mid)[0]
    #f, ax = plt.subplots(1, 1,figsize=(8,3))
    #ax.plot(np.arange(-1 * cc_win, cc_win + 1),cc)
    
    
    # Compute A1 cross-correlation with audio VOXEL
#     print('Calculating correlations...')
#     cc = np.zeros((a1_zscore.shape[0],(2 * cc_win + 1)))
#     a1_mid = a1[:,(cc_win):(-cc_win)]
#     for i in range(2 * cc_win + 1):
#         if i == 2 * cc_win:
#             audio_shift = rms_TRs[i:]
#         else:
#             audio_shift = rms_TRs[i:(-2 * cc_win + i)]
#         for v in range(a1_zscore.shape[0]):
#             cc[v,i] = pearsonr(audio_shift, a1_mid[v,:])[0]
    #f, ax = plt.subplots(1, 1,figsize=(8,3))
    #ax.plot(np.arange(-1 * cc_win, cc_win + 1),cc)
    return cc,a1,nTR


In [5]:
#old way, includes 6 motion, FD, all compcors, and all cosines
def make_conf2(conffile):
		#Create an array of regression coefficients
		conf = np.genfromtxt(conffile, names=True)
		#conf = pd.read_csv(conffile,sep = '\t')
		#Check the average framewise displacement: 
		try:
			fd = np.nan_to_num(conf['FramewiseDisplacement'])
		except ValueError:
			fd = np.nan_to_num(conf['framewise_displacement'])
		avg = fd.mean()

		#check scrub motion outliers 
		try:
			scrub = np.column_stack([conf[k] for k in conf.dtype.names if 'outlier' in k])
			if scrub.shape[1] > (0.5*scrub.shape[0]):
				print('\t',scrub.shape[1],'trs to be scrubbed')
		except ValueError:
			pass

		#6 motion parameters (want plus the differentials (12)?)
		try:
			motion = np.column_stack((conf['X'],
								  conf['Y'],
								  conf['Z'],
								  conf['RotX'],
								  conf['RotY'],
								  conf['RotZ']))
		except ValueError:
			motion_trans = np.column_stack(([conf[k] for k in conf.dtype.names if 'trans' in k and 'power' not in k]))
			motion_rot = np.column_stack(([conf[k] for k in conf.dtype.names if 'rot' in k and 'power' not in k]))
# 			motion = np.column_stack((conf['trans_x'],
# 					  conf['trans_y'],
# 					  conf['trans_z'],
# 					  conf['rot_x'],
# 					  conf['rot_y'],
# 					  conf['rot_z']))
		try:
			compcor = np.column_stack(([conf[k] for k in conf.dtype.names if 'aComp' in k]))
			cosines = np.column_stack([conf[k] for k in conf.dtype.names if 'Cosine' in k])
			csf = conf['CSF']
			wm = conf['WhiteMatter']
		except ValueError:
			compcor = np.column_stack(([conf[k] for k in conf.dtype.names if 'a_comp' in k]))
			cosines = np.column_stack([conf[k] for k in conf.dtype.names if 'cosine' in k])
			csf = conf['csf']
			wm = conf['white_matter']
		compcor = compcor[:,0:5]
		reg = np.column_stack((csf,
						   wm,
						   #fd,
						   #compcor,    
						   cosines,
						   motion_trans,
						   motion_rot, #np.vstack((np.zeros((1, motion.shape[1])),np.diff(motion, axis=0)))
							  ))

		#replace NAN with zeros if in first row
		np.nan_to_num(reg[0,:],copy = False)
		print('\tConf file is', reg.shape,'FD avg:',avg)
        #save out
		sub = conffile.split('/')[5]
		regout = '/data/musevent/confound_reg/%s_5acompcor.csv' %sub
		np.savetxt(regout, reg, delimiter=',')
		return reg

In [ ]:
#loop through subjects 
preprocfolder = '/data/musevent/fmriprep_singularity/fmriprep/'
subs = [preprocfolder+elem for elem in os.listdir(preprocfolder) if '.' not in elem and 'musevent' in elem]
subs.sort()

#audiopath = '/home/msachs/music_emo_events/fMRIMasters/'
audiopath = '/home/msachs/music_emo_events/fMRIMasters_vids/'
audio_files = [audiopath + elem for elem in os.listdir(audiopath) if 'mp4' in elem or 'wav' in elem]

trialorder = pd.read_csv('/home/msachs/music_emo_events/musevent_scan_order.csv',header = None,nrows = 54)    

print('Creating a1 mask in right space...')
mask_outname = '/home/msachs/a1_2mm2fmirprep.nii.gz'
if not os.path.exists(mask_outname):
    # Get metadata about fMRI data
    nii_template = nib.load('/data/musevent/fmriprep_singularity/fmriprep/sub-musevent02/func/sub-musevent02_task-musevent_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
    nii_shape = nii_template.header.get_data_shape()
    a1_2mm = nib.load('/home/msachs/templateflow/ROIs_for_TFA/a1plus_2mm.nii')
    #a1_mask = nib.load('/home/msachs/aud_ho_roi.nii.gz')
    aud_resamp = nilearn.image.resample_to_img(a1_2mm, nii_template,interpolation='nearest') 
    #aud_bool = np.nonzero(aud_resamp.get_data() > 0)
    mask_nifti = nib.Nifti1Image(aud_resamp.get_data(), nii_template.affine,nii_template.header)
    print('Saving',mask_outname)
    nib.save(mask_nifti,mask_outname)
else:
    aud_resamp = nib.load(mask_outname) 
aud_bool = np.nonzero(aud_resamp.get_data() > 0)

    
# Get indices for whole brain and A1
#a1_mask = nib.load('/home/msachs/aud_ho_roi.nii.gz').get_data() > 0
#a1_2mm_mask = nib.load('/home/msachs/templateflow/ROIs_for_TFA/a1plus_2mm.nii').get_fdata()
#from nilearn.input_data import NiftiLabelsMasker
#plotting.plot_roi(aud_resamp, bg_img='/data/musevent/fmriprep_singularity/fmriprep/sub-musevent02/func/sub-musevent02_task-musevent_run-2_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz')
#masker = NiftiLabelsMasker(labels_img=mask_outname, standardize=True)
#time_series = masker.fit_transform(run)


startcut = 4 #6s = 4Trs
endcut = 6 #9s = 6TRs
ignore_TRs = startcut + endcut

# Cross-correlation window size (in TRs)
cc_win = 20
cc_group = np.zeros((3,(2 * cc_win + 1),len(subs)))
#a1_group = np.zeros((nTR - ignore_TRs, len(subs)))
pieces = [elem.split('/')[5] for elem in audio_files]
pieces.sort()
nTRs = [607,607,607,607,462]

#for removing people
#subs = [elem for elem in subs if 'musevent05' not in elem]
for si,subfolder in enumerate(subs):
    sub = subfolder.split('/')[5]
    #subniftis = [subfolder + '/func/' + elem for elem in os.listdir(subfolder + '/func/') if 'bold.nii.gz' in elem and 'videvent' not in elem]
    subniftis = [subfolder + '/func/' + elem for elem in os.listdir(subfolder + '/func/') if 'bold.nii.gz' in elem]
    subniftis.sort()
    subtrials = trialorder[trialorder[0].str.contains(sub)].values[0][1:4]
    datasetout = '/data/musevent/aud_rms_check/%s_a1plus_2mm_cc_filt_nocompcor_mp4.h5' %(sub)
#     if os.path.exists(datasetout):
#         print('Data exist, loading',datasetout)
#         a1_runs = dd.io.load(datasetout)
#     else:
#         print(sub,len(subtrials),len(subniftis))
    a1_runs = {} 
    #subtrials = [elem for elem in subtrials if 'rhap' not in elem]
    for ri,run in enumerate(subniftis):
        if 'videvent' not in run:
            subtrials_mus = [elem for elem in subtrials if 'rhap' not in elem]
            conffile = [subfolder + '/func/' + elem for elem in os.listdir(subfolder + '/func/') if 'tsv' in elem and 'run-'+str(ri+1) in elem and 'videvent' not in elem][0]
            audio_file = [elem for elem in audio_files if subtrials_mus[ri].upper() in elem][0]
        else:
            conffile = [subfolder + '/func/' + elem for elem in os.listdir(subfolder + '/func/') if 'tsv' in elem and 'videvent' in elem][0]
            audio_file = [elem for elem in audio_files if 'RHAP' in elem][0]
        songname = audio_file.split('/')[5]
        songTR = nTRs[pieces.index(songname)]
        print(ri,run,audio_file,conffile,songTR,songTR-ignore_TRs)

        try: 
            #a1_runs[songname]['cc']
            cc_group[ri,:,si] = a1_runs[songname]['cc']
            #print('MADE IT TO INNER LOOP')
        except KeyError:
            #here are the functions
            rmsdata = '/home/msachs/music_emo_events/%s_rms_trs.npy' %songname
            if os.path.exists(rmsdata):
                rms_TRs = np.load(rmsdata)
            else:
                rms_TRs,rms_conv,rms = rms_reg(audio_file,songTR,ignore_TRs)
                np.save(rmsdata, rms_TRs)
            cc,a1,nTR = audcor(run,conffile,rms_TRs,ignore_TRs,cc_win)
            a1_runs[songname] = {}
            a1_runs[songname]['a1_data'] = a1
            a1_runs[songname]['rm_TRs'] = rms_TRs
            a1_runs[songname]['cc'] = cc
            cc_group[ri,:,si] = cc
            print(a1_runs[songname]['rm_TRs'].shape)
            print('\tSaving output',datasetout)
            dd.io.save(datasetout,dd.io.ForcePickle(a1_runs))
        #print('\tMAX CC:',a1_runs[songname]['cc'].max())
        print('\tMAX CC:',a1_runs[songname]['cc'].max(),cc_group[ri,:,si].max())
        
#plot results
f, ax = plt.subplots(2, math.ceil(len(subs)/2),figsize=(20,10))
plt.setp(ax, ylim=[-.3,.3])
for ri,run in enumerate(subniftis):
    for si,subfolder in enumerate(subs):
        sub = subfolder.split('/')[5]
        subtrials = trialorder[trialorder[0].str.contains(sub)].values[0][1:4]
        if 'videvent' not in run:
            subtrials_mus = [elem for elem in subtrials if 'rhap' not in elem]
            audio_file = [elem for elem in audio_files if subtrials_mus[ri].upper() in elem][0]
        else:
            audio_file = [elem for elem in audio_files if 'RHAP' in elem][0]
        songname = audio_file.split('/')[5]
        if si >= math.ceil(len(subs)/2):
            print(sub,ri,si-int(len(subs)/2),cc_group[ri,:,si].max)
            ax[1, si-math.ceil(len(subs)/2)].plot(np.arange(-1 * cc_win, cc_win + 1), cc_group[ri,:,si],label = songname)
            ax[1, si-math.ceil(len(subs)/2)].set_xlabel('fMRI vs. audio timepoint shift')
            ax[1, si-math.ceil(len(subs)/2)].set_ylabel('fMRI/audio Correlation')
            ax[1, si-math.ceil(len(subs)/2)].set_title(sub)
            ax[1, si-math.ceil(len(subs)/2)].legend(loc='lower center')
        else:
            print(sub,ri,si)
            ax[0, si].plot(np.arange(-1 * cc_win, cc_win + 1), cc_group[ri,:,si],label = songname)
            ax[0, si].set_xlabel('fMRI vs. audio timepoint shift')
            ax[0, si].set_ylabel('fMRI/audio Correlation')
            ax[0, si].set_title(sub)
            ax[0, si].legend(loc='lower center')
    f.savefig('/data/musevent/aud_rms_check/a1plus_2mm_cc_' + str(len(subs)) + 'ss.png')
    #plt.show()
    
#PAIRWISE ISC
group = {}
for ip,p in enumerate(pieces):
    songTR = nTRs[ip]
    group[p] = np.zeros((songTR-ignore_TRs,len(subs)))
    for si,subfolder in enumerate(subs):
        sub = subfolder.split('/')[5]
        datasetout = '/data/musevent/aud_rms_check/%s_a1plus_2mm_cc_filt.h5' %(sub)
        print('Loading',datasetout)
        a1_runs = dd.io.load(datasetout)
        try:
            group[p][:,si] = a1_runs[p]['a1_data']
        except KeyError:
            pass
    bools = np.all((group[p] == 0), axis=0)
    groupfilt = group[p][:,~bools]
    iscs = {}
    iscs[p] = isc(groupfilt, pairwise=False)
    runsubs = groupfilt.shape[1]
    print(ip,p,'ISC mean across',runsubs,'ss:',iscs[p].mean())

    #indivudal subject ISC: 
    nsubs = group[p].shape
    for s1 in range(runsubs):
        isc_sub = []
        for s2 in range(runsubs):
#             if s1 >= s2: 
#                 continue
            isc_sub.append(pearsonr(groupfilt[:,s1], groupfilt[:,s2])[0])
        print(ip,p,s1,np.mean(isc_sub))
        
    #print(ip,p,np.mean(isc))

Creating a1 mask in right space...
0 /data/musevent/fmriprep_singularity/fmriprep/sub-musevent01/func/sub-musevent01_task-musevent_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz /home/msachs/music_emo_events/fMRIMasters_vids/051921_A1_master.mp4 /data/musevent/fmriprep_singularity/fmriprep/sub-musevent01/func/sub-musevent01_task-musevent_run-1_desc-confounds_timeseries.tsv 607 597
Loading /home/msachs/music_emo_events/fMRIMasters_vids/051921_A1_master.mp4
Loading brain and masking...
	Conf file is (607, 27) FD avg: 0.1370996459763097
Cleaning and zscoring (607, 27)
RMS (597,) A1 (597,) num_TRs 607 0.5081930378578539
Calculating correlations...
(597,)
	Saving output /data/musevent/aud_rms_check/sub-musevent01_a1plus_2mm_cc_filt_nocompcor_mp4.h5
	MAX CC: 0.2135221617125272 0.2135221617125272
1 /data/musevent/fmriprep_singularity/fmriprep/sub-musevent01/func/sub-musevent01_task-musevent_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz /home/msachs/music_emo_events/fMRIMa

In [2]:
import nibabel as nib
from moviepy.editor import *
import numpy as np
from scipy.stats import zscore
import h5py
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from nilearn import surface
from nilearn import plotting
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker,MultiNiftiMasker

#Get auditory ROI (HO 44) in Terminal 
xmllint --format HarvardOxford-Cortical.xml
fslmaths /opt/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr0-2mm.nii.gz -thr 44 -uthr 44 /home/msachs/heschl_ho_roi.nii.gz
fslmaths /opt/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr0-2mm.nii.gz -thr 45 -uthr 45 /home/msachs/planum_ho_roi.nii.gz
fslmaths /home/msachs/heschl_ho_roi.nii.gz -add /home/msachs/planum_ho_roi.nii.gz /home/msachs/aud_ho_roi.nii.gz
fslmaths /home/msachs/aud_ho_roi.nii.gz -bin /home/msachs/aud_ho_roi.nii.gz

# #plot it
roifile = '/home/msachs/aud_ho_roi.nii.gz'
#roifile = '/home/msachs/heschl_ho_roi.nii.gz'
#roifile = '/home/msachs/planum_ho_roi.nii.gz'
plotting.plot_roi(roifile, title='HO Auditory ROI')

#set up other paths 
basepath = '/home/msachs/music_networks/'
preprocpath = os.path.join(basepath,'preprocessed')
stimulipath = os.path.join(basepath,'stimuli')
songs = [elem for elem in os.listdir(stimulipath) if '._' not in elem]
songs = [elem for elem in songs if '.D' not in elem]

# Get indices for whole brain and A1
brain_inds = np.nonzero(nib.load('/home/msachs/music_networks/avg152T1_gray.nii.gz').get_data() > 0)
a1_mask = nib.load(roifile).get_data() > 0

#Start with a single song
song = 'hnl_norm.mp3'
#task = 'sadsh'
audio_file = os.path.join(stimulipath,song)


# Hemodynamic Response Function (from AFNI)
dt = np.arange(0, 15)
p = 8.6
q = 0.547
hrf = np.power(dt / (p * q), p) * np.exp(p - dt / q)


# Don't use the very beginning of the stimulus to compute correlations, to
# ensure that the story has actually started being spoken
ignore_TRs_start = 15 #5 for scanner to start and an additional 10
ignore_TRs_end = 10 #scan continued 10 seconds after music ended 
ignore_TRs = 10

#Load the filtered NIFTI files
fpaths = []
for (dirpath, dirnames, filenames) in os.walk(preprocpath):
        fpaths += [os.path.join(dirpath, file) for file in filenames]
        
niftis = [elem for elem in fpaths if 'filt.nii.gz' in elem]

#remove
remove = ['sub-28']
for r in remove:
    niftis = [elem for elem in niftis if r not in elem] #if you want the cut ones
print(len(niftis))
fnames = [elem for elem in niftis if task in elem]
fnames.sort()
len(fnames)
nSubj = int(len(niftis)/4) #should be 39 at max
print('Number of subjects:',nSubj)

# Get metadata about fMRI data
nii_template = nib.load(fnames[0])
TR = nii_template.header['pixdim'][4]
nii_shape = nii_template.header.get_data_shape()
nTR_orig = nii_shape[3]
nTR = nTR_orig - 15

# Load audio, and calculate audio envelope regressor
nTR = 183
ignore_TRs = 15
TR = 1
print('Loading audio...')
clip = AudioFileClip(audio_file)
Fs = clip.fps
samples = clip.to_soundarray()
samples = np.mean(samples, axis=1)
T = np.floor(samples.shape[0] / Fs).astype(int)
rms = np.zeros(T)
for t in range(T):
    rms[t] = np.sqrt(np.mean(np.power(samples[(Fs * t):(Fs * (t + 1))], 2)))
rms_conv = np.convolve(rms, hrf)[:T]
rms_TRs = zscore(np.interp(np.linspace(0, (nTR - 1) * TR, nTR),
                           np.arange(0, T), rms_conv)[ignore_TRs:], ddof=1)
rms_TRs = zscore(np.interp(np.linspace(0, (nTR - 1) * TR, nTR),
                           np.arange(0, T), rms_conv)[5:-10], ddof=1)
print(len(rms),len(rms_conv),len(rms_TRs))


# Compute correlation between rms_TRs and each voxel timecourse
print('Calculating correlations...')
audio_corr = np.zeros(nii_shape[0:3] + (nSubj,))
a1 = np.zeros((nTR_orig, nSubj))
group_mean = np.zeros(nii_shape)
for s in range(nSubj):
    print('   Subj ' + str(s + 1))
    D = nib.load(fnames[s]).get_data()
    a1[:, s] = zscore(np.mean(D[a1_mask], axis=0), ddof=1)
    group_mean = group_mean + D / nSubj
    audio_corr[brain_inds + (s,)] = \
        np.matmul(rms_TRs,
                  zscore(D[brain_inds][:, ignore_TRs_start:nTR_orig - ignore_TRs_end], axis=1, ddof=1).T) \
        / (len(rms_TRs) - 1)
audio_corr[np.isnan(audio_corr)] = 0

# Compute correlations for group average timecourses
group_corr = np.zeros(nii_shape[0:3])
group_corr[brain_inds] = \
    np.matmul(rms_TRs,
              zscore(group_mean[brain_inds][:, ignore_TRs_start:nTR_orig - ignore_TRs_end],
                     axis=1, ddof=1).T) \
    / (len(rms_TRs) - 1)

# Compute A1 ISC
ISC = np.zeros(nSubj)
for loo_subj in range(nSubj):
    group = np.mean(a1[:, np.arange(nSubj) != loo_subj], axis=1)
    subj = a1[:, loo_subj]
    ISC[loo_subj] = pearsonr(group, subj)[0]
ISC = np.mean(ISC)

# Compute A1 cross-correlation with audio
a1_group = np.mean(a1, axis=1) #average signal in the auditory corte
a1_group = aud_hap['subj_data'].mean(axis = 1)
cc_win = 20
cc = np.zeros(2 * cc_win + 1)
a1_group_mid = a1_group[(ignore_TRs_start + cc_win):(- ignore_TRs_end -cc_win)] #shift brain to start from 25 (20 after music starts) and end 
for i in range(2 * cc_win + 1):
    #print(i)
    if i == 2 * cc_win:
        audio_shift = rms_TRs[i:]
    else:
        audio_shift = rms_TRs[i:(-2 * cc_win + i)]
    
    cc[i] = pearsonr(audio_shift, a1_group_mid)[0]
    if cc[i] > 0.26:
        print(i,(-2 * cc_win + i),audio_shift.shape,cc[i])
        break
print(cc.max)       
#Max correlation between auditory signal and average signal in A1 is   
#when brain data starts at 35 (30 after music starts) and auditory starts at auditory starts at 9 to 127


x = np.arange(0,a1_group_mid.shape[0])
plt.plot(x,audio_shift,'g')
plt.plot(x,a1_group_mid,'b')

a1_group_cut = a1_group[ignore_TRs_start:-ignore_TRs_end]
pearsonr(a1_group_cut,rms_TRs)

x = np.arange(0,a1_group_cut.shape[0])
#plt.plot(x,rms_TRs,'g')
plt.plot(x,a1_group_cut,'b')

x = np.arange(0,a1_group.shape[0])
plt.plot(x,a1_group,'b')

np.diff(a1_group).max()
np.where(np.diff(a1_group) == np.diff(a1_group).max()) #max differential is at 9
a1_group[9]
a1_group[20]
a1_group[-10]
rms
    
# Save files
nib.save(nib.Nifti1Image(audio_corr, affine=nii_template.affine),
         'audio_corr_' + task + '.nii.gz')

nib.save(nib.Nifti1Image(group_corr, affine=nii_template.affine),
         'audio_corr_group_' + task + '.nii.gz')

#PLot
f, ax = plt.subplots(2, 2,figsize=(10,10))
ax[0, 0].imshow(group_corr[:, :, 38], vmin=-0.3, vmax=0.3,cmap = 'seismic')
ax[0, 0].set_axis_off()
ax[0, 0].set_title('Correlation with audio (axial)')
ax[0, 1].imshow(np.flipud(group_corr[:, 41, :].T), vmin=-0.3, vmax=0.3)
ax[0, 1].set_axis_off()
ax[0, 1].set_title('Correlation with audio (coronal)')

ax[1, 0].plot(a1)
ax[1, 0].plot(a1_group, 'k', linewidth=2)
ax[1, 0].set_xlabel('Timepoints')
ax[1, 0].set_ylabel('A1 activity')
ax[1, 0].set_title('A1 ISC = ' + str(ISC))


ax[1, 1].plot(np.arange(-1 * cc_win, cc_win + 1), cc)
ax[1, 1].set_xlabel('fMRI vs. audio timepoint shift')
ax[1, 1].set_ylabel('fMRI/audio Correlation')

plt.show()

with h5py.File('a1_' + task + '.h5', 'w') as hf:
    hf.create_dataset("subj_data", data=a1)
    hf.create_dataset("ISC", data=ISC)
    hf.create_dataset("cc", data=cc)

SyntaxError: invalid syntax (<ipython-input-2-1e5850ed10d0>, line 13)

In [ ]:
## Load results
songs = ['sadsh','sadln','happy']
cc_win = 20

for s in songs:
    aud_hap = dd.io.load('/home/msachs/music_networks/a1_happy.h5')
    rms_load = np.load('/home/msachs/music_networks/rms_reg/happy.npy')
    cc_maxes = np.zeros(aud_hap['subj_data'].shape[1])
    for sub in range(aud_hap['subj_data'].shape[1]):
        sub_brain = aud_hap['subj_data'][:,sub]
        #sub_brain_cut = sub_brain[5:-10]
        #print(pearsonr(rms_TRs, sub_brain_cut)[0])
        cc = np.zeros(2 * cc_win + 1)
        
        a1_group_mid = sub_brain[(ignore_TRs_start + cc_win):(- ignore_TRs_end -cc_win)] #shift brain to start from 25 (20 after music starts) and end 
        for i in range(2 * cc_win + 1):
            #print(i)
            if i == 2 * cc_win:
                audio_shift = rms_TRs[i:]
            else:
                audio_shift = rms_TRs[i:(-2 * cc_win + i)]

            cc[i] = pearsonr(audio_shift, a1_group_mid)[0]
        cc_maxes[sub] = cc.max()
        print(cc.max())   